In [7]:
import json
import pandas as pd
from utils import prep_ans
import numpy as np
from vqa_question_type_processing import question_type_processing as vqa_question_type_processing
from simpsons_question_type_processing import question_type_processing as simpsons_question_type_processing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from io import BytesIO
from PIL import Image
import numpy as np
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import wandb

In [8]:
vqav2_anno_paths = ["/home/ngoc/data/vqav2/new_vqa2_test_annotation.json"]

In [9]:
vqa_ans_to_idx, idx_to_vqa_ans, question_type_map = json.load(open("/home/ngoc/githubs/chatgpt-hie-vqa/datasets/answer_dicts_vqav2.json", 'r'))

In [10]:

vqa_question_paths = ["/home/ngoc/data/vqav2/new_vqa2_test_question.json"]
vqa_questions = []
for vqa_question_path in vqa_question_paths:
    with open(vqa_question_path, 'r') as file:
        vqa_questions += json.load(file)
        
        
vqa_processed_questions = {}
for question in vqa_questions:
    vqa_processed_questions[question["question_id"]] = question["question"]


In [11]:
vqav2_answers = []
for vqav2_anno_path in vqav2_anno_paths:
    with open(vqav2_anno_path, 'r') as file:
        vqav2_answers += json.load(file)

In [12]:
def proc_ans(ans, ans_to_ix):
    ans_prob_dict = {}
    for ans_ in ans['answers']:
        ans_proc = prep_ans(ans_['answer'])
        if ans_proc not in ans_prob_dict:
            ans_prob_dict[ans_proc] = 1
        else:
            ans_prob_dict[ans_proc] += 1
    answer_scores = {} 
    for ans_ in ans_prob_dict:
        if ans_ in ans_to_ix:
            answer_scores[ans_] = get_score(ans_prob_dict[ans_])
    return answer_scores

def get_score(occur):
    if occur == 0:
        return .0 
    elif occur == 1:
        return round(1/3,2)
    elif occur == 2:
        return round(2/3,2)
    elif occur >= 3:
        return 1

In [13]:
qa_pairs = []
for vqav2_answer in  vqav2_answers:
    question_id = vqav2_answer["question_id"]
    multiple_choice_answer =  prep_ans(vqav2_answer['multiple_choice_answer'])
    
    if multiple_choice_answer in vqa_ans_to_idx:
        question = vqa_processed_questions[question_id]
        item = {"question": question, "answer": multiple_choice_answer}
        qa_pairs.append(item)
    # process_answers = ", ".join(list(proc_ans(vqav2_answer, vqa_ans_to_idx).keys()))
    # question = vqa_processed_questions[question_id]
    # if len(process_answers) > 0:
    #         item = {"question": question, "answer": process_answers}
    #         qa_pairs.append(item)

In [14]:
qa_pairs = pd.DataFrame(qa_pairs)

In [16]:
qa_pairs = qa_pairs.drop_duplicates()

In [17]:
def map_prompt(question, answer):

    prompt = (f'Classify a question-asnwer pair into yes/no, object, color, number, location, human, action, sport, other:\n'
                f'Question: What is the capital of France?.\n'
                f'Answer: Paris.\n'
                f'Type: location\n\n'
                f'Question: {question}.\n'
                f'Answer: {answer}.\n'
                f'Type:')
    
    return prompt

In [11]:
qa_pairs["prompt"] = qa_pairs[["question", "answer"]].apply(lambda x: map_prompt(x["question"], x["answer"]), axis =1)

In [12]:
qa_pairs.to_csv("full_vqav2_qa_prompting.csv", index=False)

In [13]:
print(qa_pairs.iloc[1000].prompt)

Classify a question-asnwer pair into yes/no, object, color, number, location, human, action, sport, other:
Question: What is the capital of France?.
Answer: Paris.
Type: location

Question: Who are riding motorcycles?.
Answer: police.
Type:


In [14]:
qa_pairs[:45000].to_csv("first_vqav2_qa_prompting.csv", index=False)

In [15]:
qa_pairs[45000:].to_csv("second_vqav2_qa_prompting.csv", index=False)

In [18]:
qa_pairs

,question,answer
0,What is this photo taken looking through?,net
1,What position is this man playing?,pitcher
2,What color is the players shirt?,orange
3,Is this man a professional baseball player?,yes
4,What color is the snow?,white
...,...,...
94491,Is it a light or heavy rain?,light
94494,Can he be sharpening blades?,yes
94495,Is this a normal thing to see in recent history?,no
94496,What links the two chains together?,ring


In [1]:
import json
import pandas as pd
qa_pair_counts = {}
file_paths = ['vqav2_qa_type_1.json', 'vqav2_qa_type_2.json', 
              'vqav2_qa_type_3.json', "new_1.json", "new_2.json",
              "new_3.json"]
vqa_test_qa_type = []
for file_path in file_paths:
    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            question = data['question']
            answer = data['answer']
            
            qa_pair = (question, answer)
            if qa_pair in qa_pair_counts:
                qa_pair_counts[qa_pair] += 1
            else:
                question_answer_type = data["question_answer_type"] if 'question_answer_type' in data else data["question_type"]
                item = {
                    "question": question,
                    "answer": answer,
                    "question_answer_type": question_answer_type
                }
                qa_pair_counts[qa_pair] = 1
                vqa_test_qa_type.append(item)
                
                

num_unique_pairs = len(qa_pair_counts)
num_duplicates = sum(count > 1 for count in qa_pair_counts.values())

print("Number of unique question-answer pairs:", num_unique_pairs)
print("Number of duplicate question-answer pairs:", num_duplicates)

Number of unique question-answer pairs: 107190
Number of duplicate question-answer pairs: 8106


In [2]:
vqa_test_qa_type = pd.DataFrame(vqa_test_qa_type)
vqa_test_qa_type = vqa_test_qa_type.rename(columns={"question_type": "qa_type"})

In [3]:
vqa_test_qa_type.to_csv("vqa_test_qa_type.csv", index=False)